In [13]:
import tensor_to_structure as ttf
import tensor_to_structure.utilities as ttf_utilities
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf

# Rerun this cell to load changes in the ttf module
import importlib
importlib.reload(ttf)
importlib.reload(ttf_utilities)

<module 'tensor_to_structure.utilities' from '/home/simon/skoltech/chemoinformatics/project/tensor_to_structure/tensor_to_structure/utilities.py'>

In [2]:
with open('data/structs', 'rb') as f:
    ligands = pickle.load(f)

In [3]:
molecules = []
for i, ligand in enumerate(ligands[:12]):
    try:
        molecule = ttf_utilities.rdkit_to_molecule(ligand)
        molecules.append(molecule)
    except ValueError as error:
        print('Bad ligand:', i, error)

Bad ligand: 3 Bad Conformer Id


In [10]:
vox = ttf_utilities.molecule_to_tensor(molecules[0])

In [41]:
k =5
max_channels = tf.math.reduce_max(vox, axis=-1)
x_dim, y_dim, z_dim = max_channels.shape
max_channels = tf.reshape(max_channels, [-1])
_, top_k_indexes = tf.math.top_k(max_channels, k=k)
new_indexes = []
for max_index in top_k_indexes:
    same_good_indexes = tf.where(max_channels == max_channels[max_index])
    same_good_indexes = tf.random.shuffle(same_good_indexes)
    for index in same_good_indexes:
        if index not in new_indexes:
            new_indexes.append(index)
            break
result_indexes = []
for index in new_indexes:
    index = int(index)
    idx_z = index % z_dim
    index //= z_dim
    idx_y = index % y_dim
    index //= y_dim
    idx_x = index
    result_indexes.append((idx_x, idx_y, idx_z))
result_indexes

[(13, 21, 21), (19, 17, 21), (21, 14, 19), (28, 27, 15), (21, 19, 20)]

array([0.        , 0.        , 0.00397454, 1.        , 1.        ])